**מטלת סיכום חלק ב**

מגישה: אלישבע אוייר 332515956

קישור לגיטהאב- https://github.com/ElishOyer/CarPricePrediction 

-בשלב הראשון של המטלה, אבחר כמה עמודות שנחשבות רלוונטיות לחיזוי המחיר על בסיס היגיון ושיקולים עסקיים


.לאחר מכן אבצע ניקוי והכנה של הנתונים על ידי מילוי ערכים חסרים בעמודות אלו, תוך שימוש בשיטת ההשלמה המתאימה ביותר עבור כל עמודה

In [17]:
from datetime import datetime
import pandas as pd
import numpy as np
import ppscore as pps
import re
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [18]:
file_path = r'C:\Users\Elisheva\Downloads\dataset.csv'  
data = pd.read_csv(file_path)

-בדיקה ראשונית על ידי בדיקת הקורלציה של כל פיצ'ר עם המחיר, במטרה לקבל תמונה ראשונית של השפעת הפיצ'רים על המחיר

.יכול לגלות קשרים שאינם לינאריים, שאותם מתאם פיירסון לא יכול לגלות ולכן בחרתי לבדוק קשרים באמצעותו PPS

In [19]:
data_cleaned = data.dropna()

# הגדרת המאפיינים הרלוונטיים
numerical_features = ['Year', 'Hand', 'Km', 'capacity_Engine', 'Supply_score', 'Pic_num']
categorical_features = ['manufactor', 'model', 'Gear', 'Engine_type', 'Area', 'City', 'Description', 'Color', 'Test', 'Prev_ownership', 'Curr_ownership']

# חישוב הקורלציה הלינארית עבור המאפיינים המספריים
correlation_matrix = data_cleaned[numerical_features + ['Price']].corr()
price_correlation = correlation_matrix['Price'].sort_values(ascending=False)
numerical_correlation = price_correlation.abs().sort_values(ascending=False).drop('Price')

# הדפסת הקורלציה הלינארית
print("קורלציות לינאריות עבור המאפיינים המספריים:")
for feature, correlation in numerical_correlation.items():
    print(f"{feature}: {correlation:.4f}")

# חישוב ה-PPS עבור המאפיינים הקטגוריאליים
pps_scores = []
for feature in categorical_features:
    pps_score = pps.score(data_cleaned, x=feature, y='Price')['ppscore']
    pps_scores.append({'feature': feature, 'pps': pps_score})

# המרת התוצאות ל-DataFrame
pps_df = pd.DataFrame(pps_scores)
pps_df = pps_df.sort_values(by='pps', ascending=False)

# הדפסת ה-PPS
print("\nPPS עבור המאפיינים הקטגוריאליים:")
for index, row in pps_df.iterrows():
    print(f"{row['feature']}: {row['pps']:.4f}")

קורלציות לינאריות עבור המאפיינים המספריים:
Year: 0.7109
Supply_score: 0.3228
Hand: 0.2481
Pic_num: 0.1305

PPS עבור המאפיינים הקטגוריאליים:
Description: 0.4839
Test: 0.4764
Color: 0.4051
City: 0.3670
Area: 0.3340
Prev_ownership: 0.1886
manufactor: 0.0000
model: 0.0000
Gear: 0.0000
Engine_type: 0.0000
Curr_ownership: 0.0000


.שמתי לב שמאפיינים שעל פי ההגיון שלי היו צריכים להיות עם קורלציה גבוהה כמו מודל, יצרן וכו (מאפיינים מרכזיים של רכב) קיבלו מתאם אפסי לכן החלטתי לבדוק מחדש אחרת

-החלטתי לבדוק עם קורלציה פשוטה גם עבור מאפיינים קטגוריאליים שעלולים להיות בעלי קשרים לא לינאריים אך כך אוכל לקבל נקודת מבט נוספת על הדאטה

In [20]:
data_cleaned = data.dropna()

categorical_features = ['manufactor', 'model', 'Gear', 'Engine_type', 'Area', 'City', 'Description', 'Color', 'Test']
data_cleaned = pd.get_dummies(data_cleaned, columns=categorical_features, drop_first=True)

correlation_matrix = data_cleaned.corr()
price_correlation = correlation_matrix['Price'].sort_values(ascending=False)
feature_correlation = {}

for feature, correlation in price_correlation.items():
    original_feature = feature.split('_')[0] if '_' in feature else feature
    if original_feature not in feature_correlation:
        feature_correlation[original_feature] = []
    feature_correlation[original_feature].append(correlation)

average_correlations = {feature: sum(correlations) / len(correlations) for feature, correlations in feature_correlation.items()}
sorted_average_correlations = sorted(average_correlations.items(), key=lambda x: abs(x[1]), reverse=True)

for feature, avg_correlation in sorted_average_correlations:
    print(f"{feature}: {avg_correlation:.4f}")

Price: 1.0000
Year: 0.7109
Supply: -0.3228
Hand: -0.2481
Engine: -0.1825
Pic: -0.1305
Gear: -0.0671
manufactor: 0.0649
model: 0.0338
Description: 0.0311
Area: -0.0226
Test: -0.0128
City: 0.0092
Color: -0.0015


-מתוך שתי השיטות אבחר בעמודות שאמשיך לחקור

.עמודות התיאור, תמונות הן עמודות שהשלמת ערכיהן החסרים לא תהיה מדויקת (לא מספריים) או לא רלוונטית ולכן נוותר עליהן מראש, למרות המתאמים הגבוהים שלהן

.עמודות הצבע, עיר ואיזור בעיתיות מבחינת השלמת ערכים אך בעלות קורלציה יחסית גבוהה ולכן אבחר להכניס את הגבוהות ביותר מבניהן להמשך חקר הדאטה

:לכן בחרתי לחקור את העמודות הבאות

manufactor, Year, model, Hand, Gear, capacity_Engine, Km, Supply_score, Color, City.

.עמודות אלו נבחרו מאחר שהן נתפסות כמשפיעות ישירות על הערך והביצועים של הרכב, ולפיכך סביר להניח שהן ישפיעו באופן משמעותי על המחיר המבוקש לרכב

In [21]:
selected_columns = ['Year', 'Hand', 'Gear', 'capacity_Engine', 'Km', 'manufactor', 'model', 'Price', 'City', 'Color']
df = data[selected_columns].copy()
#print(df.dtypes) # חלק מהמשתנים לא בסוג המבוקש ממטלה 1 ולכן בהכנה של הדאטה נתקן זאת
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             1500 non-null   int64  
 1   Hand             1500 non-null   int64  
 2   Gear             1499 non-null   object 
 3   capacity_Engine  1474 non-null   object 
 4   Km               1412 non-null   object 
 5   manufactor       1500 non-null   object 
 6   model            1500 non-null   object 
 7   Price            1500 non-null   float64
 8   City             1500 non-null   object 
 9   Color            787 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 117.3+ KB


:יש צורך בניקיון דאטה וטיפול בערכים חסרים 

.אבצע וידוא סוגי נתונים על פי המבוקש, זיהוי ערכים חריגים והמרתם ומילוי ערכים חסרים 

.מילוי על פי הערך הנפוץ ביותר, מכיוון שיש רק ערך חסר אחד, מילוי לפי הערך הנפוץ ביותר ישמור על ההומוגניות של הנתונים -gear בעמודה

.נשתמש בקיבוץ על פי יצרן ודגם ונמלא ערכים חסרים לפי חציון הקבוצה המתאימה -capacity_Engine בעמודה 

.אם ערך הקיבוץ חסר נמלא לפי החציון הכללי, מכיוון שנפח מנוע הוא נתון מספרי ושימוש בחציון מפחית את השפעת ערכים קיצוניים

.מילוי על פי ממוצע הקילומטראז' לשנה בהתאם למספר השנים שהרכב על הכביש (אם נתון השנה חסר, נשלים עם חציון) -Km בעמודה

.בעמודות הצבע בחרתי להשלים על פי הערך הנפוץ ביותר מכיוון שהשלמה זו היא עמידה בפני השפעות חיצוניות ואינה דורשת מידע נוסף או הנחות על ההתפלגות של הנתונים

.יש מעט מאוד נתונים לכן כל דרך שבה נבחר להשלים ערכים חסרים לא תביא ערכים מדויקים. נוותר על שימוש בעמודה זו עבור החיזוי -Supply_score בעמודה

In [37]:
def prepare_data(df):
    selected_columns = ['Year', 'Hand', 'Gear', 'capacity_Engine', 'Km', 'manufactor', 'model', 'City', 'Color', 'Price']
    df = df[selected_columns].copy()
    
    # שינוי ערכים של "None" ל-NaN בכל העמודות
    df.replace('None', np.nan, inplace=True)
    
    # המרת עמודות לסוגי נתונים מתאימים
    df['Gear'] = df['Gear'].astype('category')
    df['capacity_Engine'] = df['capacity_Engine'].astype(str).replace(['None', 'nan'], np.nan).str.replace(',', '').astype(float)
    df['manufactor'] = df['manufactor'].astype('category')
    df['model'] = df['model'].astype('category')
    df['City'] = df['City'].astype('category')
    df['Color'] = df['Color'].astype('category')
    df['Km'] = df['Km'].astype(str).replace(['None', 'nan'], np.nan).str.replace(',', '').astype(float)

    # מילוי ערכים חסרים 
    # Gear - מילוי לפי הערך הנפוץ ביותר
    df['Gear'] = df['Gear'].fillna(df['Gear'].mode()[0])
    # Color - מילוי לפי הערך הנפוץ ביותר
    df['Color'] = df['Color'].fillna(df['Color'].mode()[0])
    # capacity_Engine - מילוי לפי החציון
    df['capacity_Engine'] = df['capacity_Engine'].fillna(df['capacity_Engine'].median())
    # Km - מילוי לפי החציון
    df['Km'] = df['Km'].fillna(df['Km'].median())

    # ניקוי עמודת ה-model
    def clean_model(row):
        model = row['model']
        manufactor = row['manufactor']
        if manufactor in model:
            model = model.replace(manufactor, '').strip()
        if manufactor == 'Lexsus' and 'לקסוס' in model:
            model = model.replace('לקסוס', '').strip()
        # Remove year in parentheses from model if it appears
        model = re.sub(r'\(\d{4}\)', '', model).strip()
        return model

    df['model'] = df.apply(clean_model, axis=1)
    
    # חישוב הממוצעים עבור המשתנים הקטגוריאליים
    categorical_features = ['manufactor', 'model', 'Gear', 'Color', 'City']
    for feature in categorical_features:
        means = df.groupby(feature)['Price'].mean()
        df[f'{feature}_mean'] = df[feature].map(means)

    # שמירה על המאפיינים הנבחרים בלבד
    final_df = df[['manufactor', 'model', 'Gear', 'City', 'Color'] +
                  [f'{feature}_mean' for feature in categorical_features] +
                  ['Year', 'Hand', 'Km', 'capacity_Engine', 'Price']]

    return final_df

.המרת ערכים קטגוריאליים לערכים מספריים מבוססי ממוצע תעזור למודל להבין את המשמעות של הקטגוריות השונות ולתפוס את ההשפעה שלהן על מחיר הרכב

In [38]:
# הפעלת הפונקציה כדי לקבל את הדאטה פריים הנקי
df_prepared = prepare_data(df)
display(df_prepared)

,manufactor,model,Gear,City,Color,manufactor_mean,model_mean,Gear_mean,Color_mean,City_mean,Year,Hand,Km,capacity_Engine,Price
0,יונדאי,i35,אוטומטית,רעננה,כחול כהה מטאלי,60003.037037,44000.000000,52323.998511,53391.304348,48794.117647,2015,2,144000.0,1600.0,51000.0
1,ניסאן,מיקרה,אוטומטית,אבן יהודה,כחול בהיר,47581.250000,42076.923077,52323.998511,39000.000000,63800.000000,2018,1,69000.0,1200.0,49000.0
2,סוזוקי,סוויפט,אוטומטית,רמת,לבן,49667.692308,37090.909091,52323.998511,52309.224009,40875.000000,2010,1,145000.0,1450.0,22500.0
3,טויוטה,אוריס,טיפטרוניק,רחובות,אפור מטאלי,66201.031250,62350.000000,47455.555556,50010.945205,44877.142857,2016,1,27300.0,1600.0,63000.0
4,קיה,פיקנטו,אוטומטית,ראשון לציון,לבן,52316.880240,50870.384615,52323.998511,52309.224009,53207.315789,2012,1,70000.0,1248.0,37000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,סקודה,אוקטביה,אוטומטית,כפר קאסם,לבן,48529.908333,53353.333333,52323.998511,52309.224009,60000.000000,2016,2,180.0,1400.0,60000.0
1496,אלפא רומיאו,ג'ולייטה,ידנית,חיפה,לבן,48374.875000,60200.000000,38484.797468,52309.224009,57867.346939,2013,4,160000.0,1400.0,45000.0
1497,סקודה,ראפיד,אוטומטית,כפר מנדא,לבן,48529.908333,39000.000000,52323.998511,52309.224009,48333.333333,2014,1,123000.0,1400.0,30000.0
1498,ניסאן,ג'וק JUKE,אוטומטית,אשדוד,אפור מטאלי,47581.250000,44875.000000,52323.998511,50010.945205,58800.000000,2011,3,118000.0,1600.0,28000.0


In [39]:
# בדיקת ערכים חסרים ב- הדאטה פריים המעובד
print(df_prepared.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   manufactor       1500 non-null   category
 1   model            1500 non-null   object  
 2   Gear             1500 non-null   category
 3   City             1500 non-null   category
 4   Color            1500 non-null   category
 5   manufactor_mean  1500 non-null   category
 6   model_mean       1500 non-null   float64 
 7   Gear_mean        1500 non-null   category
 8   Color_mean       1500 non-null   float64 
 9   City_mean        1500 non-null   float64 
 10  Year             1500 non-null   int64   
 11  Hand             1500 non-null   int64   
 12  Km               1500 non-null   float64 
 13  capacity_Engine  1500 non-null   float64 
 14  Price            1500 non-null   float64 
dtypes: category(6), float64(6), int64(2), object(1)
memory usage: 130.4+ KB
None


.אכן טיפלנו בכל הערכים החסרים

.כעת נחלק את הדאטה ל2 סטים לפני העיבוד על מנת להבטיח את עצמאות סט הבדיקה ולתת הערכה אמיתית יותר של ביצועי המודל

.מטרתנו היא לבדוק את ביצועי המודל באופן אמיתי ככל הניתן, לכן נשתמש בחלוקה לפני העיבוד - כך נוודא שהמודל לא נחשף לאינפורמציה מיותרת בזמן האימון

In [40]:
# חלוקת לסט אימון וסט בדיקה לפני עיבודם באמצעות הפונקציה שיצרנו 
train_df, test_df, y_train, y_test = train_test_split(df, df['Price'], test_size=0.2, random_state=42)

# הכנת הנתונים באמצעות הפונקציה prepare_data
df_prepared_train = prepare_data(train_df)
df_prepared_test = prepare_data(test_df)

# הכנת הנתונים לאימון המודל
X_train_prepared = df_prepared_train.drop(columns=['Price', 'manufactor', 'model', 'Gear', 'City', 'Color'])
y_train_prepared = df_prepared_train['Price']
X_test_prepared = df_prepared_test.drop(columns=['Price', 'manufactor', 'model', 'Gear', 'City', 'Color'])
y_test_prepared = df_prepared_test['Price']

-ביצוע חלוקת הנתונים, חיפוש פרמטרים עבור האלסטיק נט ומציאת 5 המאפיינים בעלי המשקל הגבוה ביותר לחיזוי המחיר

.נעריך את ביצועי המודל על ידי חלוקה חוזרת ונשנית של הנתונים לסטי אימון ובדיקה- באמצעות קרוס ולידציה

In [46]:
# הגדרת המודל והפרמטרים לחיפוש
elastic_net = ElasticNet()
param_grid = {'alpha': [0.1, 1.0, 10.0], 'l1_ratio': [0.2, 0.5, 0.8]}
grid_search = GridSearchCV(elastic_net, param_grid, cv=10, scoring='neg_mean_squared_error')
grid_search.fit(X_train_prepared, y_train_prepared)

# המודל הטוב ביותר
best_model = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# חיזוי על סט הבדיקה
y_pred = best_model.predict(X_test_prepared)

# חמשת המאפיינים עם המשקלות הגבוהים ביותר (חיובי ושלילי)
feature_importance = pd.Series(best_model.coef_, index=X_train_prepared.columns)
top_features = feature_importance.abs().sort_values(ascending=False).head(5).index
top_features_importance = feature_importance[top_features]

print("Top 5 features with their importance:")
print(top_features_importance)

# הצגת הדאטה פריים עם חמשת המאפיינים הנבחרים והמחיר
final_df = X_test_prepared[top_features.tolist()].copy()
final_df.loc[:, 'Price'] = y_test_prepared.values
display(final_df)

Best parameters found:  {'alpha': 1.0, 'l1_ratio': 0.2}
Top 5 features with their importance:
Year               1568.848971
Hand               -106.505258
capacity_Engine       1.740971
model_mean            0.723183
Color_mean            0.532456
dtype: float64


,Year,Hand,capacity_Engine,model_mean,Color_mean,Price
1116,2018,1,1248.0,52960.000000,52993.979058,67000.0
1368,2014,1,1200.0,24000.000000,52993.979058,23000.0
422,2016,1,1600.0,58900.000000,52993.979058,69500.0
413,2018,2,1600.0,63100.000000,52993.979058,72000.0
451,2014,2,1500.0,65333.333333,52993.979058,69000.0
...,...,...,...,...,...,...
983,2013,4,2000.0,33000.000000,39583.333333,33000.0
799,2014,3,1500.0,47033.266667,53250.000000,45000.0
1265,2009,2,1468.0,34000.000000,52993.979058,22000.0
1150,2013,4,1250.0,52960.000000,52993.979058,34000.0


:מצאנו ש5 הפיצ׳רים הטובים ביותר לחיזוי המחיר הם

Year, Hand, capacity_Engine, model, Color.

.המאפיינים מבוססי ממוצע מייצגים את הממוצע של מחירי הרכב עבור הדגמים והצבעים השונים כך שהמודל השתמש במידע זה כדי לחזות מחירים בצורה מדויקת יותר

.כל המאפיינים עם משקל חיובי, חוץ מעמודת היד שעם משקל שלילי 

.במקרים רבים, מאפיינים עם קורלציות שליליות יכולים להיות חשובים מאוד בהבנת הגורמים המשפיעים על המחיר ובבניית מודל חיזוי מדויק

.ולכן נכלול את המאפיינים בעלי הערכים האבסולוטיים הגבוהים ביותר

In [47]:
# חישוב RMSE
rmse = np.sqrt(mean_squared_error(y_test_prepared, y_pred))
print(f"RMSE: {rmse}")

# ביצוע קרוס ולידציה
cross_val_rmse = -cross_val_score(best_model, X_train_prepared, y_train_prepared, cv=10, scoring='neg_mean_squared_error')
cross_val_rmse_mean = np.sqrt(cross_val_rmse.mean())
print(f"Cross-Validation RMSE: {cross_val_rmse_mean}")

RMSE: 10480.454498077275
Cross-Validation RMSE: 12679.914758998912


.הקרבה בין מדדים אלה מצביעה על ביצועים עקביים בחלוקות שונות של הנתונים, מה שמרמז על כך שהמודל מתפקד היטב גם על נתונים שלא נראו במהלך האימון

.ערכים אלו הם בטווח מקובל, מה שמעיד על כך שהמודל יכול לחזות מחירי רכבים בצורה אמינה על בסיס המאפיינים שניתנו